<a href="https://colab.research.google.com/github/NFroehl/DMML2021_Nestle/blob/main/UNIL_NESTLE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Project : Nestle

Here are the important libraries that we will use

In [ ]:
# Install and update spaCy
!pip install -U spacy

# Download the french language model
!python -m spacy download fr

# Libraries:

import pandas as pd
import spacy
import string
from spacy.lang.fr.stop_words import STOP_WORDS

     |████████████████████████████████| 6.0 MB 16.3 MB/s 
     |████████████████████████████████| 10.1 MB 38.0 MB/s 
     |████████████████████████████████| 451 kB 52.8 MB/s 
     |████████████████████████████████| 628 kB 80.1 MB/s 
     |████████████████████████████████| 42 kB 1.7 MB/s 
     |████████████████████████████████| 181 kB 69.9 MB/s 
  Attempting uninstall: catalogue
    Found existing installation: catalogue 1.0.0
    Uninstalling catalogue-1.0.0:
      Successfully uninstalled catalogue-1.0.0
  Attempting uninstall: srsly
    Found existing installation: srsly 1.0.5
    Uninstalling srsly-1.0.5:
      Successfully uninstalled srsly-1.0.5
  Attempting uninstall: thinc
    Found existing installation: thinc 7.4.0
    Uninstalling thinc-7.4.0:
      Successfully uninstalled thinc-7.4.0
  Attempting uninstall: spacy
    Found existing installation: spacy 2.2.4
    Uninstalling spacy-2.2.4:
      Successfully uninstalled spacy-2.2.4
⚠ As of spaCy v3.0, shortcuts like 'fr' are d

First, we read the Training Data using the download link.

In [ ]:
# Read the data:

# We take the url to doawnload the training data:

url = "https://storage.googleapis.com/kagglesdsdata/competitions/32066/2798629/training_data.csv?GoogleAccessId=web-data@kaggle-161607.iam.gserviceaccount.com&Expires=1638871683&Signature=BxIwsdE%2BQaVaP44mJZiajFAd05%2F2RADSA4xIKEqvZsO2ulQbUJL5TIBs5jx88hatetORhWF9annOzgcuLJYV5Do2qAcBUfz1whR60FQHgpjv%2F7X3orM8UPG2w7ZhqC0vd3fRpFWdfI%2FqZnGg6CSySAFth8Yultup9ANQl6EMy59y91dl3uC6OYgmiHHHOfOCrU6LJoez2BCER8l1rY1VIKUwJogQhx%2BNDP7kCdDzsLYmkGzwqAn49sLqulJLw%2F7t7Bm5BJclauqXS8FJzXDWzB4S1ndG43qMnQJpFHP8hc2ndPiQX8q1nZNN8NudQjZJjVK4xr1lEm56vZqGRhYrZA%3D%3D&response-content-disposition=attachment%3B+filename%3Dtraining_data.csv"
Data = pd.read_csv(url, delimiter=",")
Data.head(2)

,id,sentence,difficulty
0,0,Les coûts kilométriques réels peuvent diverger...,C1
1,1,"Le bleu, c'est ma couleur préférée mais je n'a...",A1


We get some informations about the training data in order to see how good they are.

In [ ]:
Data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4800 entries, 0 to 4799
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   id          4800 non-null   int64 
 1   sentence    4800 non-null   object
 2   difficulty  4800 non-null   object
dtypes: int64(1), object(2)
memory usage: 112.6+ KB


In [ ]:
# Base rate: the data-set is balanced
Data.difficulty.value_counts()

A1    813
C2    807
C1    798
A2    795
B1    795
B2    792
Name: difficulty, dtype: int64

Tokening the Data With spaCy

In [ ]:
# We create a list of punctuation marks
punctuations = string.punctuation
punctuations

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [ ]:
# And a list of stopwords
stop_words = spacy.lang.fr.stop_words.STOP_WORDS

#list(stop_words)

In [ ]:
# Let's implement the tokenizer function:

# Load French language model:
sp = spacy.load('fr_core_news_sm')

# Tokenizer function:
def spacy_tokenizer(sentence):
    # Create token object, which is used to create documents with linguistic annotations.
    mytokens = sp(sentence)
    
    # Lemmatize each token and convert each token into lowercase
    mytokens = [ word.lemma_.lower().strip() for word in mytokens ]

    # Remove stop words and punctuation
    mytokens = [ word for word in mytokens if word not in stop_words and word not in punctuations ]

    # Return preprocessed list of tokens
    return mytokens

Vectorization Feature Engineering (TF-IDF)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vector = TfidfVectorizer(tokenizer=spacy_tokenizer)

Classification of the reviews using logistic regression

In [ ]:
# Select features
X_train = Data['sentence'] # the features we want to analyze
y_train = Data['difficulty'] # the labels, or answers, we want to test against

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
# Define classifier
classifier = LogisticRegression()

# Create pipeline
pipe = Pipeline([('vectorizer', tfidf_vector),
                 ('classifier', classifier)])

# Fit model on training set
pipe.fit(X_train, y_train)

Pipeline(steps=[('vectorizer',
                 TfidfVectorizer(tokenizer=<function spacy_tokenizer at 0x7f6b9f987b00>)),
                ('classifier', LogisticRegression())])

To do the prediction, we download the test data as we did for the train data.

In [ ]:
url = "https://storage.googleapis.com/kagglesdsdata/competitions/32066/2798629/unlabelled_test_data.csv?GoogleAccessId=web-data@kaggle-161607.iam.gserviceaccount.com&Expires=1638891443&Signature=Yq3ALBqeX%2BRm7bWum%2FoP%2FCSO3s4iW8mcJfp07Gm11sUWLdennLm6wKgoeI%2FsCc3grJSl7cj%2B9O1JbP2l7gxwMUMH2o0zysO9zPXZODpQCG73sRgurwfRbJqQ3C%2Fv8zVCJ0D1TKaBC8vH2Mn%2FrhmFUm58RerTIk%2FdFmBEScD6nyLN591GA3gAY04NIYCLH34pRCRQ4uwYvDra%2FxYH7ABvLQ8tOwloYXHBkq%2BkssqXUcVhypJS9USj4p%2FkFWj5GwgVf8Vf3ytj2FXDTf0GBw0IEMT2bVHQRE8T1ZWc8%2BLqmdRMqA%2BwA%2BlTCUyua9FyQNokxrEd3j%2FzBNIXf8ZGD3JTDQ%3D%3D&response-content-disposition=attachment%3B+filename%3Dunlabelled_test_data.csv"
Data2 = pd.read_csv(url, delimiter=",")
Data2.head(2)

,id,sentence
0,0,Nous dûmes nous excuser des propos que nous eû...
1,1,Vous ne pouvez pas savoir le plaisir que j'ai ...


Then we do the prediction with our model.

In [ ]:
X_test = Data2["sentence"]
y_pred = pipe.predict(X_test)

In [ ]:
Y = pd.DataFrame(y_pred)

In [ ]:
Y.columns = ['id, difficulty']
Y

,"id, difficulty"
0,C1
1,B1
2,A1
3,C1
4,C2
...,...
1195,C1
1196,A2
1197,C2
1198,B2


In [ ]:
Y.to_csv('prediction.csv', header = True, index = True)
# I couldn't save directly on my files
#r'C:\Users\albon\Documents\cours M1\Data mining and machine learning\Project\prediction.csv'

The code to evaluate the model but we can't use it as our test data is not labelled.

In [ ]:
#from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
# Evaluate the model
#def evaluate(true, pred):
#    precision = precision_score(true, pred, average='micro')
#    recall = recall_score(true, pred, average='micro')
#    f1 = f1_score(true, pred, average='micro')
#    print(f"CONFUSION MATRIX:\n{confusion_matrix(true, pred)}")
#    print(f"ACCURACY SCORE:\n{accuracy_score(true, pred):.4f}")
#    print(f"CLASSIFICATION REPORT:\n\tPrecision: {precision:.4f}\n\tRecall: {recall:.4f}\n\tF1_Score: {f1:.4f}")

In [ ]:
# Predictions
#y_pred = pipe.predict(X_test)

# Evaluation - test set
#evaluate(y_test, y_pred)